In [2]:
from preprocessing import Audio
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
import librosa as li
import librosa.display
import IPython.display as ipd

%matplotlib inline
#tf.debugging.set_log_device_placement(True)
GPU = tf.config.list_physical_devices('GPU')
CPU = tf.config.list_physical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/physical_device:GPU:0'

<h1>Подготовка данных для обучения и тестирования</h1>

In [3]:
def get_data():
    def norm(x):
        std = np.std(x, ddof=1)
        return (x - np.mean(x)) / std

    PATH_0 = r'AudioData/0 Данные' # negative class
    PATH_1 = r'AudioData/0 Шум' # noise
    PATH_2 = r'AudioData/1 Мира' # positive class

    PATHS = li.util.find_files(PATH_0)[:1000] + li.util.find_files(PATH_1) + li.util.find_files(PATH_2)
    N = len(PATHS)

    np.random.shuffle(PATHS)

    DATA = []
    LABELS = []
    for i, path in enumerate(PATHS):
        sound = Audio(path)
        e_parts = sound.augmented()
        for e in e_parts: 
            DATA.append(norm(e))
            LABELS.append(sound.label)
        print('Loading {:.1f}%'.format(i / N * 100))
        ipd.clear_output(wait=True)

    return (np.array(DATA), np.array(LABELS))

In [4]:
data, labels = get_data()

Loading 99.9%


In [19]:
data.shape, labels.shape

((2411, 20), (2411,))

In [21]:
np.reshape(labels, [BATCH_SIZE, 1, 1])

array([[[0]],

       [[0]],

       [[0]],

       ...,

       [[0]],

       [[0]],

       [[0]]])

In [23]:
BATCH_SIZE = len(data)
test_data = np.reshape(data, [BATCH_SIZE, 1, 20])
test_labels = np.reshape(labels, [BATCH_SIZE, 1])

dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 1, 20), (None, 1)), types: (tf.float64, tf.int32)>

<h1>Создание модели нейронной сети</h1>

In [43]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, 20,)),
    tf.keras.layers.GRU(20, kernel_regularizer=tf.keras.regularizers.l2(0.01), reset_after=False),
    tf.keras.layers.Dense(1, activation='sigmoid')
], name='Mira_GRU-v4.1')

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.AUC(),
    ]
)

model.summary()

Model: "Mira_GRU-v4.1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, 20)                2460      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 21        
Total params: 2,481
Trainable params: 2,481
Non-trainable params: 0
_________________________________________________________________


<h1>Обучение</h1>

In [45]:
history = model.fit(
    dataset,
    epochs=10000,
    steps_per_epoch=len(dataset),
    verbose=0,
)
model.evaluate(dataset)

1/1 [==============================] - 0s 3ms/step - loss: 0.0840 - recall_8: 0.9992 - precision_8: 0.9730 - auc_7: 0.9970


[0.08398228883743286,
 0.9992063641548157,
 0.9729520678520203,
 0.9970088601112366]

In [46]:
model.save(r'models\v4\{}.h5'.format(model.name))